In [ ]:
import importlib
import data
from gurobipy import *

# get data from excel
dataframes = data.get_all_pandas()

In [ ]:
from openpyxl import Workbook
from datetime import datetime
from utils import min_to_jour

def show_solution(model):
    # print(var_dict)
    print("Objectif minimal: ", m.objVal)
    #build excel of tasks
    workbook = Workbook()
    sheet = workbook.active
    sillons_df = dataframes["sillons_df"]
    taches_df = dataframes["taches_df"]
    machines_df = dataframes["machines_df"]

    sheet["A1"] = "Id_tache"
    sheet["B1"] = "Type de tache"
    sheet["C1"] = "Jour"
    sheet["D1"] = "HDébut"
    sheet["E1"] = "Durée"
    sheet["F1"] = "Sillon"
    sheet["G1"] = "Minute"
    row = 2
    for tache in var_dict.keys():
        for sillon_id in var_dict[tache].keys():
            sheet[f"B{row}"] = tache
            sheet[f"A{row}"] = tache + "_" + sillon_id
            sheet[f"C{row}"] = min_to_jour(var_dict[tache][sillon_id].X)[0]
            sheet[f"D{row}"] = min_to_jour(var_dict[tache][sillon_id].X)[1]
            if tache in list(machines_df["Machine"]):
                sheet[f"E{row}"] = machines_df[machines_df["Machine"] == tache]["Duree "].iloc[0]
            elif tache in list(taches_df["Type de tache humaine"]): 
                sheet[f"E{row}"] = taches_df[taches_df["Type de tache humaine"] == tache]["Durée"].iloc[0]
            sheet[f"F{row}"] = sillon_id
            sheet[f"G{row}"] = var_dict[tache][sillon_id].X
            row += 1

    currentDateAndTime = datetime.now().strftime("%d-%m_%H-%M-%S")
    workbook.save(filename=f"results/{data.NAME}-{currentDateAndTime}.xlsx")
# show_solution(m)


In [ ]:
import variables
import contraintes
import importlib
import utils
importlib.reload(variables) #to remive in final .py

importlib.reload(utils)

In [ ]:
# create model
importlib.reload(contraintes) #to remive in final .py
m = Model()

# generate variables (integers)
var_dict = variables.generate_variables(m, dataframes)
# generate variables
B, OCCUPATIONS = contraintes.generate_contraintes(m, dataframes, var_dict)

final_train_task = dataframes["sillons_df"].copy()
final_train_task = final_train_task[final_train_task["LDEP"]=="WPY"]
final_train_task = final_train_task[final_train_task["JARR"]==final_train_task["JARR"].max()]
# final_train_task = final_train_task[final_train_task["HDEP"]==final_train_task["HDEP"].max()]["train_id"].iloc[0]
# final_train_task = var_dict["essai de frein départ"][final_train_task]
final_train_task = quicksum([var_dict["essai de frein départ"][temp] for temp in list(final_train_task["train_id"])])
print(final_train_task)
m.setObjective(final_train_task, GRB.MINIMIZE)
# autre idée : maximiser la moyenne de l'ecart entre frein de depart et HDEP
# aute idee minimiser occupation
m.update()
# m.printStats()
# m.write("genconstr.mps")
# m.write("test.lp")
m.optimize()
# m.write("test.lp")

show_solution(m)

Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [ ]:
##### Respect du nombre de voies de chantier #####
# print(len(OCCUPATIONS))
# print(OCCUPATIONS[0])
# print(type(OCCUPATIONS[0]))
# print(OCCUPATIONS[0].getValue())
chantiers_df = dataframes["chantiers_df"]
L = [[],[],[]]
limits = [chantiers_df[chantiers_df["Chantier"]=="WPY_REC"]["Nombre de voies"].iloc[0],
          chantiers_df[chantiers_df["Chantier"]=="WPY_FOR"]["Nombre de voies"].iloc[0],
          chantiers_df[chantiers_df["Chantier"]=="WPY_DEP"]["Nombre de voies"].iloc[0]]
for line_expr in OCCUPATIONS["DEB"]:
    v = line_expr.getValue()
    
    if v == limits[0]:
        print("DEB limite atteinte")
    if v > limits[0]:
        print("DEB limite DEPASSEE")
        
    L[0].append(v)
    
for line_expr in OCCUPATIONS["FOR"]:
    v = line_expr.getValue()
    if v == limits[1]:
        print("FOR limite atteinte")
    if v > limits[1]:
        print("FOR limite DEPASSEE")
    L[1].append(v)


for line_expr in OCCUPATIONS["DEG"]:
    v = line_expr.getValue()
    if v == limits[2]:
        print("DEG limite atteinte")
    if v > limits[2]:
        print("DEG limite DEPASSEE")
    L[2].append(v)
L


In [ ]:
# tests 
importlib.reload(contraintes)
importlib.reload(variables)
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variables(m, dataframes)
# generate variables
contraintes.generate_contraintes(m, dataframes, var_dict)

In [ ]:
show_solution(m)